In [1]:
suppressMessages(library(tidyverse))
suppressMessages(library(reshape2))
suppressMessages(library(cowplot))
suppressMessages(library(gridExtra))
suppressMessages(library(hrbrthemes))
suppressMessages(library(viridis))
suppressMessages(library(grid))
suppressMessages(library(rstatix))
suppressMessages(library(ggpubr))
suppressMessages(library(vegan))

suppressMessages(source("../0_support-files/theme_CRP-MISC.R"))

source("./fig2_functions.r")
'%ni%' <- Negate('%in%')

---
## Prepare Decon data

In [2]:
##------------------------------------
## READ AND WRANGLE DATA
##------------------------------------

## Deconvolution results
results = read.csv("../1_sample-data/cfrna_deconvolution.csv",row.names=1)

## Read in meta data
meta_data <- read.csv("../1_sample-data/STable6_cfrna-samples.csv")

meta_data$expGroup = paste(meta_data$Diagnosis,meta_data$timepoint,meta_data$group,sep=" ")
meta_data$expGroup <- gsub("other ","",meta_data$expGroup)
meta_data$expGroup <- gsub("Control_Non-inflammatory","Control",meta_data$expGroup)

## Create Plotting name
sum_tab <- meta_data %>% group_by(Diagnosis,timepoint,group) %>% count()
sum_tab <- sum_tab %>% mutate(plottingName = paste0(Diagnosis,"\n",timepoint,"\n",group,"\nn=",as.character(n)))

meta_data <- merge(meta_data,sum_tab,by=c("Diagnosis","timepoint","group"))

pn_factor_order <- c()
all_pns <- sum_tab$plottingName
pn_factor_order <- c(pn_factor_order,all_pns[grepl("Control",all_pns)])

for (i in c("COVID-19","MIS-C")){
    for (ii in c("acute","post-acute","1m","3m")){
        pn_factor_order <- c(pn_factor_order,all_pns[grepl(paste0(i,"\n",ii),all_pns)])
    }}
meta_data$plottingName <- factor(meta_data$plottingName,levels= pn_factor_order)

meta_data <- meta_data %>% filter(!is.na(plottingName))

## Merge & melt
cols <- colnames(meta_data)
IDVARS = cols[!(cols %in% colnames(results))]

df_melt <- melt(meta_data,id.vars=IDVARS)
i = length(IDVARS)
colnames(df_melt)[(i+1):(i+2)] <- c("celltype","fraction")


##------------------------------------
## GROUP AND RELABEL CELLTYPES
##------------------------------------

group = list("Myeloid Progenitor" = c("Myeloid.progenitor"),
             "Erythrocyte/Erythroid Progenitor" = c("Erythrocyte.erythroid.progenitor"),
             "Intestinal Epithelial" = c("Intestinal.tuft.cell","Intestinal.enterocyte"),
             "T Cell / NK Cell" = c("T.Cell","NK.Cell"),
            "Kidney Epithelial Cell" = c("Kidney.epithelial.cell"),
            "Endothelial Cell" = c("Endothelial.cell"),
             "Neutrophil" = c("neutrophil"),
             "Platelet" = c("Platelet"),
             "Monocyte" = c("Monocyte"),
             "Thymocyte" = c("Thymocyte")
            )

## create dataframe
df2 <- stack(group) 
colnames(df2) <- c("celltype","new_celltype")

#3 merge and fix nas
df_melt2 <- merge(df_melt,df2,all.x=TRUE)
df_melt2$new_celltype <- as.character(df_melt2$new_celltype)
df_melt_all <- df_melt2                                                     ### SAVE FOR LATER PLOTTING
df_melt2[is.na(df_melt2$new_celltype),"new_celltype"] <- "other"

## Save as main table
df_melt2$celltype <- df_melt2$new_celltype
df_melt2 <- df_melt2 %>% group_by(across(c(-fraction))) %>% summarize(fraction = sum(fraction))

df_melt <- df_melt2

## New pallette
val_vector = c('#0234C1','#740C33','#4b7a47','#FB927F','#5CFBF4','#DBB008','#86C35E','#4D5D0D','#2BBBB3','#B27DF2','#1362AB')
labels <- c('Platelet','Erythrocyte/Erythroid Progenitor','Neutrophil','Myeloid Progenitor','Intestinal Epithelial','Monocyte', 'Endothelial Cell'
            ,'Thymocyte', 'Kidney Epithelial Cell', 'T Cell / NK Cell',"other")

## REFACTOR BASED ON PREVELANCE
sum <- df_melt %>% filter(Diagnosis == "MIS-C" & timepoint == "acute") %>% group_by(celltype) %>% summarize(mean=mean(fraction))
sum <- sum[order(sum$mean,decreasing=TRUE),]

labels = sum$celltype

labels <- as.character(labels[labels != "other"])
labels <- c(labels,"other")

names(val_vector) = labels



`summarise()` has grouped output by 'celltype', 'Diagnosis', 'timepoint', 'group', 'PTID', 'cfrna_sample_id',
'severity', 'reads_sequenced', 'feature_counts', 'intron_exon_ratio', 'bias53', 'rRNA_reads', 'ivig',
'days_ivig_before_samp', 'ivig_rel_samp', 'expGroup', 'n', 'plottingName'. You can override using the
`.groups` argument.


---
## Panel B - Stacked area plots

In [3]:
##------------------------------------
## PLOT
##------------------------------------

maxY = 1
severity = FALSE

HEIGHT = 1.45
WIDTH =3.95
WIDTH =3.25

df_melt_npa <- df_melt %>% filter(timepoint != "post-acute")

all_plts <- get_all_plots(df_melt_npa,val_vector,maxY,severity)


## MIS-C

pdf(file=paste0("plots/panelB_misc.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

    lay <- rbind(c(1,2,2,3))

    all_plt <- grid.arrange(all_plts[[1]], all_plts[[2]], all_plts[[4]], layout_matrix = lay)
    print(all_plt)

dev.off()


## COVID-19

pdf(file=paste0("plots/panelB_covid.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

    lay <- rbind(c(1,2,2,3))
                 
    all_plt <- grid.arrange(all_plts[[1]], all_plts[[3]], all_plts[[4]], layout_matrix = lay)
    print(all_plt)

dev.off()

## LEGEND

pdf(file=paste0("plots/panelB_legend.pdf"),
        width=4.5,height=1.2, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

    leg_plt <- all_plts[[5]]
#     leg_plt <- leg_plt + theme(legend.title = element_blank(),
#                                     legend.text = element_text(family="Helvetica", size=6, color='black'))
    leg_plt <- grid.arrange(leg_plt)
    print(leg_plt)

dev.off()

`summarise()` has grouped output by 'plottingName'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'plottingName'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'plottingName'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'plottingName'. You can override using the `.groups` argument.


TableGrob (1 x 4) "arrange": 3 grobs
  z     cells    name           grob
1 1 (1-1,1-1) arrange gtable[layout]
2 2 (1-1,2-3) arrange gtable[layout]
3 3 (1-1,4-4) arrange gtable[layout]


png 
  2

TableGrob (1 x 4) "arrange": 3 grobs
  z     cells    name           grob
1 1 (1-1,1-1) arrange gtable[layout]
2 2 (1-1,2-3) arrange gtable[layout]
3 3 (1-1,4-4) arrange gtable[layout]


png 
  2

TableGrob (1 x 1) "arrange": 1 grobs
  z     cells    name              grob
1 1 (1-1,1-1) arrange gtable[guide-box]


png 
  2

----
## Panel C - Boxplots (sup panel A)

In [4]:
##------------------------------------
## Wrange data for boxplots
##------------------------------------

diag_contrast = c("Control discovery","COVID-19 acute discovery","MIS-C acute discovery","MIS-C acute validation")

diag_df <- df_melt_all %>% 
    filter((timepoint == "acute" | Diagnosis == "Control_Non-inflammatory")) %>% 
    filter(group %in% c("discovery","validation")) %>% 
           mutate(new_celltype = ifelse(is.na(new_celltype),as.character(celltype),new_celltype)) %>% 
           mutate(celltype = new_celltype)

diag_df$plottingName <- droplevels(diag_df$plottingName)


##------------------------------------
## SET PALETTES
##------------------------------------

diag_df$expGroup = paste(diag_df$Diagnosis,diag_df$timepoint,diag_df$group,sep=" ")
diag_df$expGroup <- gsub("other ","",diag_df$expGroup)

expGroupPalette = c('#2BC0B3','#5CB2EB','#F0484E','#FBE77C')
names(expGroupPalette) <- c("MIS-C acute validation","MIS-C acute discovery","COVID-19 acute discovery","Control_Non-inflammatory Not-hospitalized discovery")

DiagnosisPalette = c('#0234C1','#740C33','#4b7a47')
names(DiagnosisPalette) <- c("MIS-C","COVID-19","Control")

##------------------------------------
## Main figure
##------------------------------------
WIDTH = 1.9
HEIGHT = 1.28

set.seed(42)


ct = "Endothelial Cell"
YLIM = 0.54
WIDTH = 1.9

pdf(file=paste0("plots/panelC_",ct,".pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

    final_plt <- make_boxplot(diag_df,ct,YLIM,expGroupPalette)+
                scale_x_discrete(guide = guide_axis(n.dodge=2))+
scale_y_continuous(breaks = c(0,0.18,0.36,0.54)) + 
    coord_cartesian(ylim = c(0,YLIM))

    print(final_plt)

dev.off()


ct = "Neutrophil"
YLIM = 0.54
WIDTH = 1.9

pdf(file=paste0("plots/panelC_",ct,".pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

    final_plt <- make_boxplot(diag_df,ct,YLIM,expGroupPalette)+
                scale_x_discrete(guide = guide_axis(n.dodge=2))+
scale_y_continuous(breaks = c(0,0.18,0.36,0.54)) + 
    coord_cartesian(ylim = c(0,YLIM))

    print(final_plt)
dev.off()


ct = "Schwann.cell"
YLIM = 0.035
WIDTH = 1.953

pdf(file=paste0("plots/panelC_",ct,".pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

    final_plt <- make_boxplot(diag_df,ct,YLIM,expGroupPalette)+
                scale_x_discrete(guide = guide_axis(n.dodge=2))+
scale_y_continuous(breaks = c(0,0.012,0.024,0.036)) + 
    coord_cartesian(ylim = c(0,YLIM))

    print(final_plt)
dev.off()

##------------------------------------
## Supplemental figure
##------------------------------------

ct = "Kidney Epithelial Cell"
YLIM = 0.035
WIDTH = 1.94
    
pdf(file=paste0("plots/SupPanelA_",ct,".cfrna.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

    final_plt <- make_boxplot(diag_df,ct,YLIM,expGroupPalette)+
                scale_x_discrete(guide = guide_axis(n.dodge=2))+
    coord_cartesian(ylim = c(0,YLIM))
    print(final_plt)
dev.off()


ct = "Thymocyte"
YLIM = 0.03
WIDTH = 1.94
    
pdf(file=paste0("plots/SupPanelA_",ct,".cfrna.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

    final_plt <- make_boxplot(diag_df,ct,YLIM,expGroupPalette)+
                scale_x_discrete(guide = guide_axis(n.dodge=2))+
    coord_cartesian(ylim = c(0,YLIM))
    print(final_plt)
dev.off()


ct = "Platelet"
YLIM = 1.3
WIDTH = 1.9

pdf(file=paste0("plots/SupPanelA_",ct,".cfrna.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

    final_plt <- make_boxplot(diag_df,ct,YLIM,expGroupPalette)+
                scale_x_discrete(guide = guide_axis(n.dodge=2))+
    coord_cartesian(ylim = c(0,YLIM))
    print(final_plt)
dev.off()


## GI 

diag_df <- df_melt_all %>% 
    filter((timepoint == "acute" | Diagnosis == "Control_Non-inflammatory")) %>% 
    filter(group %in% c("discovery","validation")) %>% 
           mutate(new_celltype = ifelse(is.na(new_celltype),as.character(celltype),new_celltype)) #%>% 
#            mutate(celltype = new_celltype)

diag_df$expGroup = paste(diag_df$Diagnosis,diag_df$timepoint,diag_df$group,sep=" ")
diag_df$expGroup <- gsub("other ","",diag_df$expGroup)


ct = "Intestinal.secretory.cell"
YLIM = 0.0027
WIDTH = 2.01

pdf(file=paste0("plots/SupPanelA_",ct,".cfrna.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

    final_plt <- make_boxplot(diag_df,ct,YLIM,expGroupPalette)+
                scale_x_discrete(guide = guide_axis(n.dodge=2))+
scale_y_continuous(breaks = c(0,YLIM/3,(YLIM/3)*2,YLIM)) + 
    coord_cartesian(ylim = c(0,YLIM*1.05))

    print(final_plt)
dev.off()


ct = "Intestinal.enterocyte"
YLIM = 0.03
WIDTH = 1.94

pdf(file=paste0("plots/SupPanelA_",ct,".cfrna.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

    final_plt <- make_boxplot(diag_df,ct,YLIM,expGroupPalette)+
                scale_x_discrete(guide = guide_axis(n.dodge=2))+
scale_y_continuous(breaks = c(0,YLIM/3,(YLIM/3)*2,YLIM)) + 
    coord_cartesian(ylim = c(0,YLIM*1.05))

    print(final_plt)
dev.off()


ct = "Intestinal.tuft.cell"
YLIM = 0.12
WIDTH = 1.94
    
pdf(file=paste0("plots/SupPanelA_",ct,".cfrna.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

    final_plt <- make_boxplot(diag_df,ct,YLIM,expGroupPalette)+
                scale_x_discrete(guide = guide_axis(n.dodge=2))+
scale_y_continuous(breaks = c(0,YLIM/3,(YLIM/3)*2,YLIM)) + 
    coord_cartesian(ylim = c(0,YLIM*1.05))

    print(final_plt)
dev.off()


## SOLID ORGAN

diag_contrast = c("Control discovery","COVID-19 acute discovery","MIS-C acute discovery","MIS-C acute validation")
diag_df_all <- df_melt_all %>% filter((timepoint == "acute" | Diagnosis == "Control_Non-inflammatory")) %>% filter(group %in% c("discovery","validation"))
diag_df_all$plottingName <- droplevels(diag_df_all$plottingName)

diag_df_all$expGroup = paste(diag_df_all$Diagnosis,diag_df_all$timepoint,diag_df_all$group,sep=" ")
diag_df_all$expGroup <- gsub("other ","",diag_df_all$expGroup)

cells <- diag_df_all %>% pull(celltype) %>% unique()
solid <- c('Intrahepatic.cholangiocyte','Secretory.cell','Respiratory.ciliated.cell','Ionocyte.luminal.epithelial.cell.of.mammary.gland',
           'Fibroblast.mesenchymal.stem.cell','Respiratory.secretory.cell','Basal.cell','Endothelial.cell','Stromal.cell','Adventitial.cell','Gland.cell',
           'Salivary.bronchial.secretory.cell','Pericyte.cell','Pancreatic.Stellate.cell','Pancreatic.alpha.beta.cell','Basal.prostate.cell',
           'Prostate.epithelia','Salivary.gland.cell','Intestinal.enterocyte','Intestinal.secretory.cell','Intestinal.tuft.cell','Type.II.Pneumocyte',
           'Cell.of.skeletal.muscle','Tendon.cell','Mesothelial.cell','Kidney.epithelial.cell')

diag_df_all_solid <- diag_df_all %>% filter(celltype %in% all_of(solid)) %>% 
    select(-celltype) %>% select(-new_celltype) %>% group_by(across(c(-fraction))) %>% 
    summarize(fraction = sum(fraction)) %>%
    mutate(celltype="solid")


ct = "solid"
YLIM = 0.7
WIDTH = 1.9
    
pdf(file=paste0("plots/SupPanelA_",ct,".cfrna.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

    final_plt <- make_boxplot(diag_df_all_solid,ct,YLIM,expGroupPalette)+
                scale_x_discrete(guide = guide_axis(n.dodge=2))+
    coord_cartesian(ylim = c(0,YLIM))
    print(final_plt)
dev.off()

# A tibble: 4 × 13
  .y.      group1      group2    n1    n2 statistic       p   p.adj p.adj.signif
  <chr>    <chr>       <chr>  <int> <int>     <dbl>   <dbl>   <dbl> <chr>       
1 fraction Control_No… COVID…    13    18        79 1.35e-1 0.18    ns          
2 fraction Control_No… MIS-C…    13    35        78 2.91e-4 0.00116 **          
3 fraction COVID-19 a… MIS-C…    18    35       193 2.1 e-2 0.042   *           
4 fraction MIS-C acut… MIS-C…    35    21       411 4.7 e-1 0.47    ns          
# … with 4 more variables: y.position <dbl>, groups <named list>, xmin <dbl>,
#   xmax <dbl>


png 
  2

# A tibble: 4 × 13
  .y.      group1    group2    n1    n2 statistic        p    p.adj p.adj.signif
  <chr>    <chr>     <chr>  <int> <int>     <dbl>    <dbl>    <dbl> <chr>       
1 fraction Control_… COVID…    13    18        38 1   e- 3 1.33e- 3 **          
2 fraction Control_… MIS-C…    13    35         4 1.24e-10 4.96e-10 ****        
3 fraction COVID-19… MIS-C…    18    35       130 3.35e- 4 6.7 e- 4 ***         
4 fraction MIS-C ac… MIS-C…    35    21       247 4.2 e- 2 4.2 e- 2 *           
# … with 4 more variables: y.position <dbl>, groups <named list>, xmin <dbl>,
#   xmax <dbl>


png 
  2

# A tibble: 4 × 13
  .y.    group1 group2    n1    n2 statistic     p p.adj p.adj.signif y.position
  <chr>  <chr>  <chr>  <int> <int>     <dbl> <dbl> <dbl> <chr>             <dbl>
1 fract… Contr… COVID…    13    18        91 0.312 0.401 ns               0.0284
2 fract… Contr… MIS-C…    13    35       102 0.003 0.012 *                0.0313
3 fract… COVID… MIS-C…    18    35       193 0.021 0.042 *                0.0370
4 fract… MIS-C… MIS-C…    35    21       418 0.401 0.401 ns               0.0428
# … with 3 more variables: groups <named list>, xmin <dbl>, xmax <dbl>


...

# A tibble: 4 × 13
  .y.    group1 group2    n1    n2 statistic     p p.adj p.adj.signif y.position
  <chr>  <chr>  <chr>  <int> <int>     <dbl> <dbl> <dbl> <chr>             <dbl>
1 fract… Contr… COVID…    13    18       127 0.708 0.708 ns                0.210
2 fract… Contr… MIS-C…    13    35       306 0.07  0.28  ns                0.230
3 fract… COVID… MIS-C…    18    35       382 0.214 0.428 ns                0.270
4 fract… MIS-C… MIS-C…    35    21       343 0.687 0.708 ns                0.311
# … with 3 more variables: groups <named list>, xmin <dbl>, xmax <dbl>


png 
  2

`summarise()` has grouped output by 'Diagnosis', 'timepoint', 'group', 'PTID', 'cfrna_sample_id', 'severity',
'reads_sequenced', 'feature_counts', 'intron_exon_ratio', 'bias53', 'rRNA_reads', 'ivig',
'days_ivig_before_samp', 'ivig_rel_samp', 'expGroup', 'n'. You can override using the `.groups` argument.


# A tibble: 4 × 13
  .y.      group1      group2    n1    n2 statistic       p   p.adj p.adj.signif
  <chr>    <chr>       <chr>  <int> <int>     <dbl>   <dbl>   <dbl> <chr>       
1 fraction Control_No… COVID…    13    18        63 3.1 e-2 4.13e-2 *           
2 fraction Control_No… MIS-C…    13    35        68 9.32e-5 3.73e-4 ***         
3 fraction COVID-19 a… MIS-C…    18    35       192 2   e-2 4   e-2 *           
4 fraction MIS-C acut… MIS-C…    35    21       421 3.73e-1 3.73e-1 ns          
# … with 4 more variables: y.position <dbl>, groups <named list>, xmin <dbl>,
#   xmax <dbl>


png 
  2

---
## Sup Panel B - Severity

In [5]:
##------------------------------------
## Prep data and measure for differences
##------------------------------------

diag_df <- df_melt_all %>% 
    filter((timepoint == "acute" & Diagnosis == "MIS-C")) %>% 
    filter(group %in% c("discovery","validation")) %>%
    mutate(group = ifelse(severity == 2, "Moderate MIS-C",
                            ifelse(severity == 3,"Severe MIS-C","other")))

## Calculate differences between groups
CELLTYPES = diag_df$celltype %>% unique()

output_list = list()

for (ct in CELLTYPES){
 mod <- diag_df %>% filter(celltype == ct) %>% filter(severity == 2) %>% pull(fraction)
    sev <- diag_df %>% filter(celltype == ct) %>% filter(severity == 3) %>% pull(fraction)
    
    t <- wilcox.test(x = mod, y = sev)
    
    output_list[[ct]] <- c(ct,t$p.value)
    
}

ct_df <- do.call("rbind", output_list) %>% data.frame() %>% rename(celltyep = X1, pval = X2)

ct_df$adj.p <- p.adjust(ct_df$pval,method = "BH")

ct_df %>% arrange(adj.p) %>% head()

##------------------------------------
## PLOT 
##------------------------------------

set.seed(42)

expGroupPalette = c('#008176','#0000a7')
names(expGroupPalette) <- c("Moderate MIS-C","Severe MIS-C")

WIDTH = 1.6
HEIGHT = 1.5

WIDTH = 1
HEIGHT = 1.28


CELLTYPE = "Endothelial.cell"
YLIM = 0.6

pdf(file=paste0("plots/SupPanelB_severity-",CELLTYPE,".pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

bxplt <- make_severity_plt(diag_df,CELLTYPE) + scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM)) + coord_cartesian(ylim = c(0,YLIM*1.05))                                         
print(bxplt)

dev.off()


CELLTYPE = "neutrophil"
YLIM = 0.6

pdf(file=paste0("plots/SupPanelB_severity-",CELLTYPE,".pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

bxplt <- make_severity_plt(diag_df,CELLTYPE) + scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM)) + coord_cartesian(ylim = c(0,YLIM*1.05))
print(bxplt)

dev.off()


CELLTYPE = "Schwann.cell"
YLIM = 0.03

pdf(file=paste0("plots/SupPanelB_severity-",CELLTYPE,".pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

bxplt <- make_severity_plt(diag_df,CELLTYPE) + scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM)) + coord_cartesian(ylim = c(0,YLIM*1.05))
print(bxplt)

dev.off()


CELLTYPE = "T.Cell"
YLIM = .15

pdf(file=paste0("plots/SupPanelB_severity-",CELLTYPE,".pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

bxplt <- make_severity_plt(diag_df,CELLTYPE) + scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM)) + coord_cartesian(ylim = c(0,YLIM*1.05))
print(bxplt)

dev.off()

Warning message in wilcox.test.default(x = mod, y = sev):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(x = mod, y = sev):
“cannot compute exact p-value with ties”


,celltyep,pval,adj.p
,<chr>,<chr>,<dbl>
Endothelial.cell,Endothelial.cell,0.0110512461120239,0.2550522
T.Cell,T.Cell,0.0124415725937463,0.2550522
Intrahepatic.cholangiocyte,Intrahepatic.cholangiocyte,0.182815025033057,0.7250338
Monocyte,Monocyte,0.224449503726726,0.7250338
Fibroblast.mesenchymal.stem.cell,Fibroblast.mesenchymal.stem.cell,0.126312255488565,0.7250338
neutrophil,neutrophil,0.136393406245089,0.7250338


png 
  2

png 
  2

png 
  2

png 
  2

---
## Sup Panel C Post-acute analysis

In [9]:
df_melt_plt <- df_melt  %>% 
    filter(timepoint  %in% c("acute","post-acute","Not-hospitalized"))

labels <- names(val_vector)

df_melt_plt <- df_melt_plt[which(df_melt_plt$celltype %in% names(val_vector)),]

##---------------------------
# Create master dataframe
cntrl_df <- df_melt_plt

##---------------------------
# Summarize
cntrl_sum <- cntrl_df %>% group_by(plottingName,celltype) %>% summarize(mean_fraction = mean(fraction))

##---------------------------
# Plot area plot

HEIGHT= 2.03
WIDTH = 2.37
ct = "post-acute"

pdf(file=paste0("plots/SupPanelC_",ct,".cfrna.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)


decon_plot <- cntrl_sum %>%
mutate(celltype=factor(celltype,levels=labels)) %>%
ggplot(aes(x=plottingName,y=mean_fraction,fill=celltype))+
geom_bar(position="stack",stat="identity", colour="black",size = 0.15)+
#         theme_minimal()+
theme_prevail()+
theme(plot.margin=grid::unit(c(1,0,0,0), "mm"),
     legend.position = "none")+
scale_fill_manual(values=val_vector)+
scale_y_continuous(position = "left",labels=scaleFUN)+ 
coord_cartesian(ylim = c(0,maxY))+
theme(axis.title.y = element_blank(),
     axis.title.x = element_blank())

decon_plot
dev.off()


`summarise()` has grouped output by 'plottingName'. You can override using the `.groups` argument.


png 
  2

---
## Panel D & E - diversity

In [11]:
##------------------------------------
# Prep the data
##------------------------------------

samples <- meta_data %>% filter(!is.na(plottingName))
res <- unique(results[which(rownames(results) %in% samples$cfrna_sample_id),])

expGroupPalette = c('COVID-19 acute discovery' = '#F0484E', 'MIS-C acute discovery' = '#5CB2EB', 'Control Not-hospitalized discovery' = '#FBE77C',"MIS-C acute validation" = '#2BC0B3',
                  'MIS-C post-acute discovery' = '#5CB2EB', 'MIS-C 1m discovery' = '#5CB2EB','MIS-C 3m+ discovery' = '#5CB2EB',
                    'COVID-19 post-acute discovery' = '#F0484E','COVID-19 1m discovery' = '#F0484E', 'COVID-19 3m+ discovery' = '#F0484E')

DiagnosisPalette = c('#0234C1','#740C33','#4b7a47')
names(DiagnosisPalette) <- c("MIS-C","COVID-19","Control")

##------------------------------------
# SIMPSON 
##------------------------------------

method = "simpson"

alpha <- data.frame(as.matrix((vegan::diversity(x=res, index = method))))
rownames(alpha) <- rownames(res)
colnames(alpha) <- method
alpha <- merge(alpha,samples,by.x=0,by.y="cfrna_sample_id")
alpha[is.na(alpha$severity),"severity"] <- 0
alpha[grepl("ontrol",alpha$severity),"severity"] <- 0
alpha <- alpha %>% filter(timepoint != "post-acute")

# PLOT

HEIGHT = 0.95
WIDTH = 1.7

pdf(file=paste0("plots/panelD_misc.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
YLIM = 1.45
SAMP_LEVELS = c('Control Not-hospitalized discovery','MIS-C acute validation','MIS-C acute discovery','MIS-C 1m discovery','MIS-C 3m+ discovery')
make_alpha_plot(alpha,SAMP_LEVELS,method)+ coord_cartesian(ylim = c(0,YLIM)) + scale_y_continuous(breaks = seq(0, 1, by = .25))
dev.off()

pdf(file=paste0("plots/panelD_covid19.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
YLIM = 1.45
SAMP_LEVELS = c('Control Not-hospitalized discovery','COVID-19 acute discovery','COVID-19 1m discovery', 'COVID-19 3m+ discovery')
make_alpha_plot(alpha,SAMP_LEVELS,method)+ coord_cartesian(ylim = c(0,YLIM)) + scale_y_continuous(breaks = seq(0, 1, by = .25))
dev.off() 

##------------------------------------
# BRAY CURTIS
##------------------------------------

# Make comparisons
bray_curtis <- data.frame(as.matrix((vegan::vegdist(x=res, method = "bray"))))
rownames(bray_curtis) <- rownames(res)
colnames(bray_curtis) <- rownames(res)

# Melt dataframe for easy comparison
bc_melt <- melt(as.matrix(bray_curtis))
colnames(bc_melt) <- c("Sam.ID_1","Sam.ID_2","BC")

## Filter out comparisons of the same samples
bc_melt <- bc_melt[bc_melt$Sam.ID_1 != bc_melt$Sam.ID_2,]

# Add diagnostic information
diag_df <- samples[,c("cfrna_sample_id","expGroup")]

## Sam.ID_1
bc_melt <- merge(bc_melt,diag_df,by.x="Sam.ID_1",by.y="cfrna_sample_id")
colnames(bc_melt)[4] <- "Diagnosis_1"

## Sam.ID_2
bc_melt <- merge(bc_melt,diag_df,by.x="Sam.ID_2",by.y="cfrna_sample_id")
colnames(bc_melt)[5] <- "Diagnosis_2"

# Add comparison variable
bc_melt$Group <- ifelse(bc_melt$Diagnosis_1 == bc_melt$Diagnosis_2,bc_melt$Diagnosis_1,"Different")

bc_melt$Group2 <- apply(X=bc_melt, MARGIN=1, FUN=
                function(x){
                  a <- x['Diagnosis_1']
                  b <- x['Diagnosis_2']
                  c<- sort(c(a,b))
                  d <- paste0(c[1],"+", c[2])
                  return(d)
                })

bc_melt$name <- apply(X=bc_melt, MARGIN=1, FUN=
                function(x){
                  a <- x['Sam.ID_2']
                  b <- x['Sam.ID_1']
                  c<- sort(c(a,b))
                  d <- paste0(c[1],"+", c[2])
                  return(d)
                })

bc_melt <- bc_melt %>% filter(!grepl("post-acute",Group2))

## PLOT

HEIGHT = 0.95
WIDTH = 1.7

method = "Bray-Curtis"

pdf(file=paste0("plots/panelE_misc.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

YLIM = 1.3

SAMP_LEVELS = c('Control Not-hospitalized discovery+Control Not-hospitalized discovery','Control Not-hospitalized discovery+MIS-C acute validation','Control Not-hospitalized discovery+MIS-C acute discovery','Control Not-hospitalized discovery+MIS-C 1m discovery','Control Not-hospitalized discovery+MIS-C 3m+ discovery')
make_bc_plot(bc_melt,SAMP_LEVELS) + coord_cartesian(ylim = c(0,YLIM)) + scale_y_continuous(breaks = seq(0, 1, by = .25))

dev.off()

               
pdf(file=paste0("plots/panelE_covid19.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

YLIM = 1.25

SAMP_LEVELS = c('Control Not-hospitalized discovery+Control Not-hospitalized discovery','Control Not-hospitalized discovery+COVID-19 acute discovery','Control Not-hospitalized discovery+COVID-19 1m discovery', 'Control Not-hospitalized discovery+COVID-19 3m+ discovery')
make_bc_plot(bc_melt,SAMP_LEVELS) + coord_cartesian(ylim = c(0,YLIM)) + scale_y_continuous(breaks = seq(0, 1, by = .25))

dev.off() 

##------------------------------------
# POST-ACUTE
##------------------------------------

method = "simpson"

alpha <- data.frame(as.matrix((vegan::diversity(x=res, index = method))))
rownames(alpha) <- rownames(res)
colnames(alpha) <- method
alpha <- merge(alpha,samples,by.x=0,by.y="cfrna_sample_id")
alpha[is.na(alpha$severity),"severity"] <- 0
alpha[grepl("ontrol",alpha$severity),"severity"] <- 0
alpha <- alpha   %>% 
    filter(timepoint  %in% c("acute","post-acute","Not-hospitalized"))

## PLOT
HEIGHT = 0.95
WIDTH = 1.4

pdf(file=paste0("plots/SupPanelD_misc.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
YLIM = 1.3
SAMP_LEVELS = c('Control Not-hospitalized discovery','MIS-C acute validation','MIS-C acute discovery','MIS-C post-acute discovery')
make_alpha_plot(alpha,SAMP_LEVELS,method)+ coord_cartesian(ylim = c(0,YLIM)) + scale_y_continuous(breaks = seq(0, 1, by = .25))
dev.off()

pdf(file=paste0("plots/SupPanelD_covid19.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
YLIM = 1.3
SAMP_LEVELS = c('Control Not-hospitalized discovery','COVID-19 acute discovery','COVID-19 post-acute discovery')
make_alpha_plot(alpha,SAMP_LEVELS,method)+ coord_cartesian(ylim = c(0,YLIM)) + scale_y_continuous(breaks = seq(0, 1, by = .25))
dev.off() 

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.



png 
  2

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.



png 
  2

# A tibble: 4 × 13
  .y.   group1       group2    n1    n2 statistic        p    p.adj p.adj.signif
  <chr> <chr>        <chr>  <int> <int>     <dbl>    <dbl>    <dbl> <chr>       
1 BC    Control Not… Contr…   156   546      9352 4.55e-50 9.1 e-50 ****        
2 BC    Control Not… Contr…   156   910     12644 1.39e-60 5.56e-60 ****        
3 BC    Control Not… Contr…   156   208     15772 6.49e- 1 6.49e- 1 ns          
4 BC    Control Not… Contr…   156   338     29524 3.2 e- 2 4.27e- 2 *           
# … with 4 more variables: y.position <dbl>, groups <named list>, xmin <dbl>,
#   xmax <dbl>


...

png 
  2

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.



png 
  2

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.



png 
  2